In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import statsmodels.api as sm
%matplotlib inline

In [2]:
df_grades = pd.read_csv('data/Grades-Table 1.csv')#
df_demographics = pd.read_csv('data/Demographics-Table 1.csv')#
df_student_aff1 = pd.read_csv('data/Student Affective 1-Table 1.csv')#
df_student_aff2 = pd.read_csv('data/Student Affective 2-Table 1.csv')#
df_study1 = pd.read_csv('data/Study Habits 1-Table 1.csv')#
df_study2 = pd.read_csv('data/Study Habits 2-Table 1.csv')#

# Feature engineering, some averageing, dummy variables, drop columns, etc.

In [3]:
# drop some columns from grades that I don't need
df_grades = df_grades[['Section','Instructor','Time','Treatment','Student',\
                       'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT','FINAL_RAW_BASE_GRADE']]

# make some categorical variable columns
df_grades = pd.get_dummies(df_grades, columns = ['Section','Time','Treatment'])

# make some categorical variables columns
df_demographics = pd.get_dummies(df_demographics, columns = ['STEM_Major', 'Ethnicity', 'sexual', 'edu_lvl'])

## Linear Regression with Grades, Demographics, Student Affective 2, and Study Habits 2

In [4]:
# merging all of the tables together to make one super table
df_merged = df_grades.merge(df_demographics, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
                                                right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
# df_merged = df_merged.merge(df_student_aff1, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
#                                                 right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
df_merged = df_merged.merge(df_student_aff2, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
                                                right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
# df_merged = df_merged.merge(df_study1, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
#                                                 right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
df_merged = df_merged.merge(df_study2, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
                                                right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
df_merged = df_merged.dropna() # all tables merged and rows with missing data have been removed



#df_grades = pd.get_dummies(df_grades, columns = ['Section','Time','Treatment'])
df_merged = pd.get_dummies(df_merged, columns = ['Instructor'])
df_merged.reset_index(drop=True, inplace=True) # just resets the indexes after all that dropping
df_merged = df_merged.drop('SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', axis=1)
df_merged.head()

,Student,FINAL_RAW_BASE_GRADE,Section_A,Section_B,Section_C,Section_D,Section_E,Section_F,Section_G,Section_H,...,Inst_?s,Peer_Comm,connections,Instructor_1.0,Instructor_2.0,Instructor_3.0,Instructor_4.0,Instructor_5.0,Instructor_6.0,Instructor_7.0
0,1.0,0.9250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.0,0.8700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3.0,0.8025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4.0,0.8765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5.0,0.9090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
X = df_merged.drop('FINAL_RAW_BASE_GRADE', axis=1).astype(float)
X['const'] = 1
y = df_merged.FINAL_RAW_BASE_GRADE

In [6]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     FINAL_RAW_BASE_GRADE   R-squared:                       0.679
Model:                              OLS   Adj. R-squared:                  0.510
Method:                   Least Squares   F-statistic:                     4.010
Date:                  Sun, 10 Jul 2016   Prob (F-statistic):           6.10e-09
Time:                          14:39:21   Log-Likelihood:                 179.26
No. Observations:                   140   AIC:                            -260.5
Df Residuals:                        91   BIC:                            -116.4
Df Model:                            48                                         
Covariance Type:              nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Student            -0.0034      0.002     -2.092      0.039      -0.007      -0.000
Section_A           0.0141      0.012      1.133      0.260      -0.011       0.039
Section_B          -0.0002      0.019     -0.010      0.992      -0.039       0.038
Section_C           0.0443      0.013      3.306      0.001       0.018       0.071
Section_D           0.0298      0.013      2.279      0.025       0.004       0.056
Section_E           0.0793      0.015      5.452      0.000       0.050       0.108
Section_F           0.0388      0.012      3.200      0.002       0.015       0.063
Section_G           0.0195      0.017      1.147      0.254      -0.014       0.053
Section_H           0.0494      0.015      3.210      0.002       0.019       0.080
Section_I          -0.0340      0.021     -1.642      0.104      -0.075       0.007
Section_J           0.0303      0.015      2.000      0.049       0.000       0.060
Time_A              0.1185      0.018      6.753      0.000       0.084       0.153
Time_N              0.1078      0.014      7.465      0.000       0.079       0.136
Time_P              0.0451      0.015      2.926      0.004       0.014       0.076
Treatment_F         0.1199      0.020      5.875      0.000       0.079       0.160
Treatment_T         0.1515      0.022      6.946      0.000       0.108       0.195
Age                -0.0047      0.004     -1.083      0.282      -0.013       0.004
prior_20            0.0060      0.023      0.265      0.791      -0.039       0.051
prior_grade         0.0283      0.050      0.560      0.577      -0.072       0.129
BIOL1010           -0.0231      0.031     -0.741      0.461      -0.085       0.039
prior_10           -0.0175      0.031     -0.567      0.572      -0.079       0.044
STEM_Major_0        0.0712      0.021      3.423      0.001       0.030       0.112
STEM_Major_1        0.0802      0.021      3.810      0.000       0.038       0.122
STEM_Major_2        0.1200      0.029      4.202      0.000       0.063       0.177
Ethnicity_A         0.0430      0.085      0.504      0.615      -0.126       0.212
Ethnicity_AA       -0.0354      0.093     -0.381      0.704      -0.220       0.149
Ethnicity_C         0.0228      0.080      0.286      0.776      -0.135       0.181
Ethnicity_H         0.0217      0.081      0.269      0.788      -0.139       0.182
Ethnicity_ME        0.0634      0.095      0.669      0.505      -0.125       0.252
Ethnicity_PI        0.0438      0.103      0.424      0.672      -0.161       0.249
sexual_0            0.1464      0.024      6.033      0.000       0.098       0.195
sexual_1            0.1249      0.021      6.022      0.000       0.084       0.166
edu_lvl_1           0.0653      0.021      3.084      0.003       0.023       0.107
edu_lvl_2           0.0688      0.015